# Elastic Pendulum

Simulation of an elastic pendulum combining spring and pendulum dynamics.

You can also find this example as a standalone Python file in the [GitHub repository](https://github.com/milanofthe/pathsim/blob/master/examples/example_elastic_pendulum.py).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Apply PathSim docs matplotlib style
plt.style.use('../pathsim_docs.mplstyle')

from pathsim import Simulation, Connection
from pathsim.blocks import ODE, Function, Scope
from pathsim.solvers import RKBS32

In [ ]:
# Initial conditions
r0, vr0 = 2, 0.0
phi0, omega0 = 0.3*np.pi, 0.0

# Physical parameters
g = 9.81          # gravity [m/s^2]
l0 = 1.0          # natural spring length [m]
k = 50.0          # spring constant [N/m]
m = 1.0           # mass [kg]
c_r = 0.3         # radial damping [kg/s]
c_phi = 0.1       # angular damping [N m s]

$$
\dot{r} = v_r \\
$$

$$
\dot{v_r} = -\frac{k}{m} (r - l_0) - \frac{c_r}{m} v_r + \omega^2 r + g \cos(\varphi)
$$

In [ ]:
# Define the radial ODE (spring-mass-damper with coupling terms)
def rad_ode(x, u, t):
    r, vr = x
    omega, phi = u
        
    # radial acceleration terms
    centrifugal = r * omega**2
    spring = -(k/m) * (r - l0)
    gravity_rad = g * np.cos(phi)
    damping = -(c_r/m) * vr
    
    accel_r = centrifugal + spring + gravity_rad + damping
    
    return np.array([vr, accel_r])

rad = ODE(rad_ode, np.array([r0, vr0]))

For the angular component we have the following odes:

$$
\dot{\varphi} = \omega
$$

$$
\dot{\omega} = - \frac{g}{r} \sin(\varphi) - \frac{2}{r} v_r \omega - \frac{c_\varphi}{r^2 m} \omega
$$

In [ ]:
# Define the angular ODE (pendulum with coupling terms)
def ang_ode(x, u, t):
    phi, omega = x
    r, vr = u
    
    # angular acceleration terms
    gravity_torque = -(g / r) * np.sin(phi)
    coriolis = -(2 / r) * vr * omega
    damping = -(c_phi / (m * r**2)) * omega
    
    accel_phi = gravity_torque + coriolis + damping
    
    return np.array([omega, accel_phi])

ang = ODE(ang_ode, np.array([phi0, omega0]))

In [ ]:
# Cartesian conversion
@Function
def crt(r, phi):
    return r*np.sin(phi), -r*np.cos(phi)

In [ ]:
# Scopes for visualization
sc1 = Scope(labels=["r [m]", "vr [m/s]", "phi [rad]", "omega [rad/s]"])
sc2 = Scope(labels=["x [m]", "y [m]"], sampling_period=0.005)

In [ ]:
blocks = [rad, ang, crt, sc1, sc2]

In [ ]:
connections = [
    Connection(ang[1], rad[0]),           # omega -> rad input 0
    Connection(ang[0], rad[1], crt[1]),   # phi -> rad input 1
    Connection(rad[0], ang[0], crt[0]),   # r -> ang input 0
    Connection(rad[1], ang[1]),           # vr -> ang input 1
    Connection(rad[:2], sc1[:2]),         # r, vr -> scope
    Connection(ang[:2], sc1[2:4]),        # phi, omega -> scope
    Connection(crt[:2], sc2[:2])
    ]

In [ ]:
# Simulation instance
sim = Simulation(
    blocks,
    connections,
    Solver=RKBS32
    )

In [ ]:
# Run the simulation
sim.run(10)

In [ ]:
# Plot state variables
sc1.plot();

We can clearly see the mass jumping back and forth, driventhrough the contracting spring at each swing:

In [ ]:
# Plot the trajectory

t, (x, y) = sc2.read()

fig, ax = plt.subplots()
ax.plot(0, 0, "o", c="k")
ax.plot(x, y)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_aspect(1)